In [ ]:
import pandas as pd
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import dill as pickle

In [ ]:
data = pd.read_excel('model/dataset.xlsx')

In [ ]:
data.head()

In [ ]:
transaction = data['Transaction']

In [ ]:
type(transaction)

In [ ]:
# Convert text to lower case:
transaction = transaction.apply(lambda X: X.lower())

In [ ]:
# Remove Numbers
r = re.compile('[0-9]+')
transaction = transaction.apply(lambda X: re.sub(r, '', X))

In [ ]:
# Remove punctuation
transaction = transaction.apply(lambda X: ''.join(c for c in X if c not in punctuation))

In [ ]:
transaction

In [ ]:
#is based on The Porter Stemming Algorithm
stopword = stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()

def lem(text):
    word_tokens = word_tokenize(text)
    lemmatized_word = [wordnet_lemmatizer.lemmatize(word, 'v') for word in word_tokens]
    return lemmatized_word

transaction = transaction.apply(lambda X: lem(X))
transaction

In [ ]:
transaction = transaction.apply(lambda X: [word for word in X if word not in stopword])
transaction

In [ ]:
def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, stop_words='english', lowercase=False)
features = tfidf.fit_transform(transaction)

In [ ]:
le = preprocessing.LabelEncoder()
labels = data['Type']
le.fit(labels)

In [ ]:
le.classes_

In [ ]:
labels = le.transform(labels)

In [ ]:
features = np.asarray(features.todense())

In [ ]:
clf = LogisticRegression(random_state=0).fit(features, labels)

In [ ]:
clf.score(features, labels)

In [ ]:
pickle.dump(le, open('model/encoder.sav', 'wb'))
pickle.dump(clf, open('model/model.sav', 'wb'))

In [ ]:
data_target = features, labels
pickle.dump(data_target, open('model/data_target.sav', 'wb'))

In [ ]:
pickle.dump(tfidf, open('model/tfidf.sav', 'wb'))

In [ ]:
pickle.dump(identity_tokenizer, open('model/identity_tokenizer.sav', 'wb'))